![gold LOGO](https://economictimes.indiatimes.com/thumb/msid-66701484,width-1200,height-900,resizemode-4,imgsize-148822/gold-4-ts.jpg?from=mdr)



# CAMADA GOLD


In [0]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pytz
from pyspark.sql.functions import lit
from pyspark.sql import Window
from pyspark.sql.functions import *
from datetime import datetime , timedelta

import requests
import json

'''
Essa configuração permite que o Delta Lake faça automaticamente a evolução do esquema 
ao detectar mudanças no esquema dos dados durante operações de merge, update ou append, 
sem a necessidade de redefinir manualmente o esquema.
'''
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
'''
Essa configuração permite que o Delta Lake faça automaticamente a união de arquivos pequenos 
durante operações de escrita, reduzindo a fragmentação sem a necessidade de otimizações manuais frequentes
'''
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
'''
Esse comando abaixo possibilita Time zone de São Paulo
'''
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

## PARÂMETROS

In [0]:
try:
  time_file = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y-%m-%dT%H:%M:%S')
  # DeltaTable.isDeltaTable(spark, gold_delta_path)

  #### Nome  e caminho onde será feita a escrita da da tabela Delta  ##########
  db = 'gold'
  table = 'ibge_news'
  gold_delta_table = f'{db}.{table}'
  gold_delta_path = 'dbfs:/mnt/gold/'
  DT_END = datetime.now().date()

  #   DT_START = '2020-01-01'

  if DeltaTable.isDeltaTable(spark,gold_delta_path):
    DT_START = spark.sql("select cast(trunc(to_date(max(left(dateIngestion,10) ),'yyyy-MM-dd'),'MM') as string) from delta.`dbfs:/mnt/gold/` ").collect()[0][0]
  else:
    DT_START = spark.sql("select cast(trunc(to_date(min(left(dateIngestion,10 ) ),'yyyy-MM-dd'),'MM') as string) from delta.`dbfs:/mnt/silver/` ").collect()[0][0]

  ### Sistema de origem ####
  source_delta_table = 'silver.ibge_news'
  
  print("##----------------------------##")
  print(f"Data e Horário de execusão             ===>  {time_file} \n")
  print(f"DT_START                               ===>  {DT_START} ")
  print(f"DT_END                                 ===>  {DT_END}")
  print(f"Banco de Dados                         ===>  {db}")
  print(f"Nome da Tabela                         ===>  {table}")
  print(f"Tabela gold a ser criada             ===>  {gold_delta_table}")
  print(f"Caminho da tabela gold a ser criada  ===>  {gold_delta_path} \n")
  print("##----------------------------##")
  print(f"Tabela de Origem                     ===> {source_delta_table}")

except Exception as e:
  print(e)

##----------------------------##
Data e Horário de execusão             ===>  2025-02-14T09:23:28 

DT_START                               ===>  2025-01-01 
DT_END                                 ===>  2025-02-14
Banco de Dados                         ===>  gold
Nome da Tabela                         ===>  ibge_news
Tabela gold a ser criada             ===>  gold.ibge_news
Caminho da tabela gold a ser criada  ===>  dbfs:/mnt/gold/ 

##----------------------------##
Tabela de Origem                     ===> silver.ibge_news


In [0]:
class GoldIngestion:
  def __init__(self,source_delta_table,gold_delta_table,gold_delta_path,dtstart,dtend):
    self.source_delta_table = source_delta_table
    self.gold_delta_table = gold_delta_table
    self.gold_delta_path = gold_delta_path
    self.dtstart = dtstart
    self.dtend = dtend
    self.dt = str(dtend).replace('-','')[0:6]

  #### CRIANDO ESTRUTURA BASE DA TABELA DELTA (CAMADA GOLD) ####
  def create_structured_delta_schema(self):

    if not DeltaTable.isDeltaTable(spark,self.gold_delta_path):
      print('Criando estrutura tabela Delta....\n')
      schema = StructType([

                      StructField('referenceMonthDate',DateType(),True,metadata={"comment": "Data do Mes de referencia"}),
                      StructField('id',LongType(),True,metadata={"comment": "Identificador único da notícia"}),
                      StructField('newsHighlight', StringType(),True,metadata={"comment": "Destaque"}),
                      StructField('editorials', StringType(),True,metadata={"comment": "Editorial"}),
                      StructField('images', StringType(),True,metadata={"comment": "Descrição das imagens"}),
                      StructField('publicationDate', TimestampType(),True,metadata={"comment": "Data da publicaçao"}),
                      StructField('introduction', StringType(),True,metadata={"comment": "Introduçao da Noticia"}),
                      StructField('link', StringType(),True,metadata={"comment": "Link da Noticia"}),
                      StructField('product_id', StringType(),True,metadata={"comment": "ID do produto"}),
                      StructField('products', StringType(),True,metadata={"comment": "Produto"}),
                      StructField('relatedProducts', StringType(),True,metadata={"comment": "Produtos relacionados"}),
                      StructField('type', StringType(),True,metadata={"comment": "Tipo "}),
                      StructField('title', StringType(),True,metadata={"comment": "Título da Noticia"}),
                      StructField('dateIngestion', TimestampType(),True,metadata={"comment": "Data de Ingestão"}),
                      StructField('dt', StringType(),True,metadata={"comment": "Data para partiçao"})
                            ])
      ### Criando estrutura tabela Delta ###
      df =  spark.createDataFrame(data= [],schema=schema)
      print(f'Criando estrutura da tabela delta no caminho .. {self.gold_delta_path}')
      df.write.format('delta').partitionBy('dt').save(f'{self.gold_delta_path}')
      
      print(f'Tabela ===> {self.gold_delta_table}  ja foi anteriormente criada no caminho ===> {self.gold_delta_path}')
      ### Criando Data base  ###
      sql = f""" CREATE DATABASE IF NOT EXISTS {db} """
      spark.sql(sql)
      ### Criando tabela de metadadados db gold  ###
      sql =  f""" DROP TABLE IF EXISTS {self.gold_delta_table}"""
      spark.sql(sql)
      print(sql)

      sql =  f""" CREATE TABLE IF NOT EXISTS {self.gold_delta_table} USING DELTA LOCATION '{self.gold_delta_path}' """
      spark.sql(sql)
      print(sql,'\n')

      print(f'Tabela {self.gold_delta_table}  criada com sucesso !!!\n')
      
     
    else:
      print(f'Tabela ===> {self.gold_delta_table}  ja foi anteriormente criada no caminho ===> {self.gold_delta_path}')
      ### Criando Data base  ###
      sql = f""" CREATE DATABASE IF NOT EXISTS {db} """
      spark.sql(sql)
      ### Criando tabela de metadadados db gold  ###
      sql =  f""" DROP TABLE IF EXISTS {self.gold_delta_table}"""
      spark.sql(sql)
      print(sql)

      sql =  f""" CREATE TABLE IF NOT EXISTS {self.gold_delta_table} USING DELTA LOCATION '{self.gold_delta_path}' """
      spark.sql(sql)
      print(sql,'\n')

      print(f'Tabela {self.gold_delta_table}  criada com sucesso !!!\n')

  #### MUDANDO O DATATYPE DOS CAMPOS IDENTIFICADOS DE STRING PARA MAP(JSON) ####
  def campos_json(self):
    try:
      print('Iniciando ... campos_json')
      df_silver = spark.table(f'{self.source_delta_table}').filter(substring(col('dateIngestion'),1,10).cast('date').between(f'{self.dtstart}',f'{self.dtend}') )
      ### Vericar com antecedencia na tabela silver campos com estrutura json ###(Não ocorre erro caso a lista estiver vazia)
      list_columns = ['imagens']
      # list_columns = []

      if len(list_columns) > 0:
        for column in df_silver.dtypes:
          if column[0] in list_columns:
            print(f'Criando estrutura JSON do campo == > {column[0]}')
            df_silver = df_silver.withColumn(f'{column[0]}_dict',from_json(f'{column[0]}',MapType(StringType(),StringType())))
            return  df_silver
      else:
        df_silver = spark.table(f'{self.source_delta_table}').filter(substring(col('dateIngestion'),1,10).cast('date').between(f'{self.dtstart}',f'{self.dtend}') )
        return df_silver
    except Exception as error:
      raise ValueError(f"{error}")

  #### FAZENDO A SEPARAÃO DOS CAMPOS QUE ESTÃO ANINHADOS EM UM CAMPO JSON, PARA CAMPOS INDIVIDUAIS ####
  def newColumns(self):
    ### Faz a separação de todos os campos dentro da do campo Json identificado  ###
    from pyspark.sql.functions import col
    df = self.campos_json()
    try:
      print('Iniciando ... newColumns')
      all_new_columns = []
      for column in df.dtypes:
        if column[1].startswith('map'):
          print(column[0],column[1],'\n')
          max_columns = df.select(max(map_keys(column[0])) ).collect()[0][0]
          all_new_columns.extend([col(f'{column[0]}.'+ cols).alias(f'{cols}') for cols in max_columns])


      return all_new_columns ,df
    
    except Exception as error:
      raise ValueError(f"{error}")

  #### SELECIONANDO OS CAMPOS PARA GRAVAÇAO DA TABELA DELTA. E ADICIONANDO OS CAMPOS QUE ESTAVAM COMO JSON PARA CAMPOS ÚNICOS E INDIVIDUAIS(all_new_columns) ####
  def transform_gold(self):
     all_new_columns,df = self.newColumns()

     df_final = (df.withColumn('dt',lit(f"{self.dt}"))
                   .withColumn('dateIngestion',current_timestamp())
                   .select( 'referenceMonthDate'
                            ,'id'
                            ,'newsHighlight'
                            ,'editorials'
                            ,'images'
                            ,'publicationDate'
                            ,'introduction'
                            ,'link'
                            ,'product_id'
                            ,'products'
                            ,'relatedProducts'
                            ,'type'
                            ,'title'
                            ,'dateIngestion'
                            ,'dt'
                            ,*all_new_columns
                         )
                 )

     print('Inicio gravação tabela delta...\n')
     (DeltaTable.forPath(spark, self.gold_delta_path).alias("old")
       .merge(df_final.alias("new"),"old.id = new.id and old.dt = new.dt")
       .whenMatchedUpdateAll()
       .whenNotMatchedInsertAll()
       .execute()
      )
     print('Gravação finalizada com sucesso!!!')
     
  
  def save_gold(self):
    self.create_structured_delta_schema()
    self.transform_gold()


In [0]:
gold_class = GoldIngestion(source_delta_table,gold_delta_table,gold_delta_path,DT_START,DT_END)
# df = gold_class.save_gold()
gold_class.save_gold()

Criando estrutura tabela Delta....

Criando estrutura da tabela delta no caminho .. dbfs:/mnt/gold/
Tabela ===> gold.ibge_news  ja foi anteriormente criada no caminho ===> dbfs:/mnt/gold/
 DROP TABLE IF EXISTS gold.ibge_news
 CREATE TABLE IF NOT EXISTS gold.ibge_news USING DELTA LOCATION 'dbfs:/mnt/gold/'  

Tabela gold.ibge_news  criada com sucesso !!!

Iniciando ... campos_json
Criando estrutura JSON do campo == > imagens
Iniciando ... newColumns
imagens_dict map<string,string> 

Inicio gravação tabela delta...

Gravação finalizada com sucesso!!!


In [0]:
%sql
select * from gold.ibge_news limit 5

referenceMonthDate,id,newsHighlight,editorials,images,publicationDate,introduction,link,product_id,products,relatedProducts,type,title,dateIngestion,dt,image_intro,float_intro,image_intro_alt,image_intro_caption,image_fulltext,float_fulltext,image_fulltext_alt,image_fulltext_caption
2017-04-01,9401,false,sociais,null,2017-04-28T07:54:00-03:00,"Indicador / Período Jan - Fev - Mar2017 Out - Nov - Dez2016 Jan - Fev - Mar2016 Taxa de desocupação 13,7% 12,0% 10,9% Rendimento real habitual R$ 2.110 R$$ 2.064 R$ 2.059 Valor do rendimento real habitual em relação a: estável estável A taxa de...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/9401-pnad-continua-taxa-de-desocupacao-vai-a-13-7-no-trimestre-encerrado-em-marco-de-2017.html,9171,9171|Divulgação mensal#pnadc1|pesquisa-nacional-por-amostra-de-domicilios-continua-mensal|2511,9171,Release,"PNAD Contínua: taxa de desocupação vai a 13,7% no trimestre encerrado em março de 2017",2025-02-14T09:23:37.347-03:00,202502,images/agenciadenoticias/00000032854504152017453428948368.JPG,,,,,,,
2017-04-01,9403,false,economicas,null,2017-04-27T08:11:00-03:00,"Março de 2017 0,09% Fevereiro de 2017 -0,45% Março de 2016 -1,20% Acumulado do ano -0,05% Acumulado em 12 meses 2,85% Em março, os preços da Indústria Geral variaram, em média, 0,09% em relação a fevereiro, acima do observado na comparação entre...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/9403-indice-de-precos-ao-produtor-de-marco-foi-de-0-09.html,9282,9282|Índice de Preços ao Produtor - Indústrias Extrativas e de Transformação|indice-de-precos-ao-produtor-industrias-extrativas-e-de-transformacao|2081,9282,Release,"Índice de Preços ao Produtor de março foi de 0,09%",2025-02-14T09:23:37.347-03:00,202502,images/agenciadenoticias/00000032812704132017275927578538.JPG,,,,,,,
2017-05-01,9406,false,economicas,null,2017-05-02T23:07:00-03:00,"Período Produção industrial Março 2017 / Fevereiro 2017 -1,8% Março 2017 / Março 2016 1,1% Acumulado em 2017 0,6% Acumulado em 12 meses -3,8% Média móvel trimestral -0,7% Em março de 2017, a produção industrial nacional mostrou redução de 1,8% frente a...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/9406-producao-industrial-cai-1-8-em-marco.html,0,,,Release,"Produção industrial cai 1,8% em março",2025-02-14T09:23:37.347-03:00,202502,images/agenciadenoticias/00000032883905142017395003315045.JPG,,,,,,,
2017-04-01,9413,false,economicas,null,2017-04-20T06:45:00-03:00,"PERÍODO TAXA Abril 0,21% Março 0,15% Abril 2016 0,51% Acumulado no ano 1,22% Acumulado em 12 meses 4,41% O Índice Nacional de Preços ao Consumidor Amplo 15 (IPCA-15) teve variação de 0,21% em abril e ficou acima da taxa de 0,15% de março em 0,06 ponto...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/9413-em-abril-ipca-15-fica-em-0-21.html,9260,9260|Índice Nacional de Preços ao Consumidor Amplo 15|indice-nacional-de-precos-ao-consumidor-amplo-15|2213,9260,Release,"Em abril, IPCA-15 fica em 0,21%",2025-02-14T09:23:37.347-03:00,202502,images/00000032751704112017170920839529.png,,,,,,,
2017-04-01,9416,false,economicas,null,2017-04-12T07:13:00-03:00,"Período Varejo Varejo Ampliado Volume de vendas Receita nominal Volume de vendas Receita nominal Fevereiro/Janeiro -0,2 0,1 1,4 1,0 Média móvel trimestral* 1,0 0,5 1,5 1,0 Fevereiro 2017 / Fevereiro 2016 -3,2 0,4 -4,2 -1,7 Acumulado 2017 -2,2 2,1 -2,1...",http://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/9416-vendas-no-varejo-variam-0-2-em-fevereiro-em-relacao-a-janeiro-2.html,9227,9227|Pesquisa Mensal de Comércio|pesquisa-mensal-de-comercio|2075,9227,Release,"Vendas no varejo variam -0,2% em fevereiro em relação a janeiro",2025-02-14T09:23:37.347-03:00,202502,images/agenciadenoticias/00000032692604102017262412627846.png,,,,,,,
